In [381]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import datetime as dt
import pyarrow
import random

from pathlib import Path
import os
from collections import deque
from src.config.paths import ROOT_DIR, SAMPLE_DIR, PRICES_DIR, META_DIR, PROCESSED_PRICES

import src.fileutils as files
import src.visualization as viz
import src.process as process
from src import process_prices

from src.process_files import PriceProcessor, RawPriceProcessor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [382]:
stations_info_file = ROOT_DIR / 'data' / 'stations.csv'
sample_file_location = SAMPLE_DIR
sample_price_location = SAMPLE_DIR / 'prices'

RSEED = 42

In [383]:
np.random.seed(RSEED)
import inspect

dus_stations_data = pd.read_csv(SAMPLE_DIR / 'stations' / 'stations_dus_plus.csv')
dus_stations = dus_stations_data.uuid

# Create a dataframe
df = pd.DataFrame({
    'datetime': [dt.datetime.now() - dt.timedelta(days=i) for i in range(10)],
    'string': ['string' + str(i) for i in range(10)],
    'integer': np.random.randint(0, 100, size=10),
    'float1': np.random.rand(10),
    'float2': np.random.rand(10),
})


# class return_one():
#     def __init__(self, data: pd.DataFrame, method=None):
#         self.one = 1
#         self.data = data
#         self.predefined_methods = {
#             'hourly': process_prices.make_hourly
#         }
#         self.set_method(method)
     
#     def set_method(self, method,):

#         if method is None:
#             self.method = None

#         elif type(method) == str:
#             if method not in self.predefined_methods:
#                 raise ValueError(f"{method} is not a a predefined method.")
#             self.method = self.predefined_methods[method]
        
#         elif inspect.isfunction(method):
#             params = inspect.signature(method).parameters.values()
#             if len(params) != 1:
#                 raise ValueError(f"{method} is required to take only 1 parameter but {len(params)} were given.")
#             self.method = method

#         else:
#             raise ValueError("Passed object is not a function or a predefined method")
    
#     def process_data(self, data: pd.DataFrame, method=None, **kwargs) -> pd.DataFrame:

#         if method:
#             self.set_method(method, **kwargs)
#         if self.method:
#             data = self.method(data, **kwargs)

#         else:
#             raise ValueError("The method process_data requires a method to be set, but None was given.")
#         return data
    
def some_function(data: pd.DataFrame)->pd.DataFrame:

    data = data.assign(
        float1 = lambda x: x['float1']*10
    )
    return data

def some_other_function(data: pd.DataFrame, multiplier = 10, add = 3)->pd.DataFrame:
    data = data.assign(
        float1 = lambda x: x['float1'] * multiplier + add
    )
    return data

subset = {
    'subset':dus_stations_data.uuid,
    'subset_column': 'station_uuid',

}
dus_stations_data = pd.read_csv(SAMPLE_DIR / 'stations' / 'stations_dus_plus.csv')

rawprocessor = RawPriceProcessor(PRICES_DIR, (PROCESSED_PRICES / 'test'), **subset)



In [384]:
one = 'data/prices/2021/08/2021-08-29-prices.csv'
two = 'data/prices/2021/08/2021-08-30-prices.csv'
three = 'data/prices/2021/08/2021-08-31-prices.csv'

rawprocessor.process_file(one)
display(rawprocessor.last_closing_prices.date.dt.date.value_counts())
rawprocessor.process_file(two)
display(rawprocessor.last_closing_prices.date.dt.date.value_counts())
rawprocessor.process_file(three)
display(rawprocessor.last_closing_prices.date.dt.date.value_counts())

date
2021-08-29    101
Name: count, dtype: int64

date
2021-08-30    106
Name: count, dtype: int64

date
2021-08-31    105
2021-08-30      1
Name: count, dtype: int64

In [385]:
display(rawprocessor.last_closing_prices.date.dt.date.value_counts())

date
2021-08-31    105
2021-08-30      1
Name: count, dtype: int64

In [386]:
rawprocessor.last_processed.groupby(level='station').tail(1).reset_index(level='date')

,date,diesel,e5,e10
station,,,,
00041414-208c-4444-8888-acdc00000414,2021-08-31 23:19:08+02:00,1.379,1.579,0.000
005056ba-7cb6-1ed2-bceb-7e82e4910d2a,2021-08-31 23:19:08+02:00,1.349,1.579,1.519
005056ba-7cb6-1ed2-bceb-7ef561844d2a,2021-08-31 23:19:08+02:00,1.369,1.609,1.549
005056ba-7cb6-1ed2-bceb-80c585ca6d2b,2021-08-31 23:19:08+02:00,1.369,1.609,1.549
005056ba-7cb6-1ed2-bceb-82ea369c0d2d,2021-08-31 23:19:08+02:00,1.369,1.609,1.549
...,...,...,...,...
fa6624d4-7bb9-4b17-9e56-31e0040428d1,2021-08-31 23:19:08+02:00,1.409,1.649,1.589
fbcdf8a7-b6ba-4ec3-ac4c-dde2f0f29934,2021-08-31 23:19:08+02:00,1.369,1.609,1.549
fcdaddc5-7dc1-49f9-8286-71e8664f9e17,2021-08-31 23:19:08+02:00,1.329,1.559,1.499


In [387]:
dus_stations_data = pd.read_csv(SAMPLE_DIR / 'stations' / 'stations_dus_plus.csv')
dus_stations = dus_stations_data.uuid
target_index = 'station'

type(dus_stations)



pandas.core.series.Series

In [388]:
from src.config.paths import PRICES_DIR, PROCESSED_PRICES
from src.config.paths import STATIONS_DIR, PROCESSED_STATIONS
from src.config.paths import META_DIR, SAMPLE_DIR

dus_stations_data = pd.read_csv(SAMPLE_DIR / 'stations' / 'stations_dus_plus.csv')
dus_stations = dus_stations_data.uuid

In [389]:
# data_processed\prices\2021\08\2021-08-29-prices.csv
# data_processed\prices\2021\08\2021-08-30-prices.csv
# data_processed\prices\2021\08\2021-08-31-prices.csv

In [390]:
# Features:
# relative prices
# opening hours + dummies
# holiday dummies
# school-holyday dummies
# crude oil
# with bins: change count/hour

# Meta
# Average Price per day (per product)
# Trade Frequency
# was this a holiday
# was this a schoolholiday
# year
# month
# day
# weekday
# average crude oil price that day

# Processing
# bin dates
# Split into 3 prices (more data but faster processing maybe?)
# make additional features independent  at first


In [391]:
# ADD PRICE CHANGES PER DAY FOR EACH STATION TO THE CLOSING TABLE
# CONVERT DATE TO ONLY DAY-DATE
# APPEND TO THE EXISTING 'CLOSING_PRICES.CSV'
from pathlib import Path
import pandas as pd
import os
from collections import deque

def save_closing_prices(df, file_path, date='date'):
    file_path = Path(file_path)
    
    # If the file doesn't exist, write the DataFrame to a new CSV file
    if not file_path.is_file():        
        df.to_csv(file_path, index=True)

    # If it does exist, compare the last line of the CSV File with the last line of the DataFrame df
    else:
        with open(file_path, "r") as file:
            last_line = deque(file, 1)[0]

        # Making sure the lines format is comparable 
        # CURRENTLY ONLY WORKS WITH DATE ON COLUMN INDEX 1
        old_timestamp = pd.to_datetime(last_line.split(',')[1])
        new_timestamp = pd.to_datetime(df[date].max())
        
        # If the new data is not already in the CSV File, append the DataFrame and safe the CSV file.
        if new_timestamp <= old_timestamp:
            print("Some data already exists in the CSV file. Data was not appended.")
        else:
            df.to_csv(file_path, mode='a', header=False, index=True)

In [392]:
def get_meta_table(df):
    # if csv exists, just open that
    # if not create a new pd.DataFrame
    # add #observations
    # add 
    pass

In [393]:
# CREATE A METATABLE WITH DAILY SUMMARY:
# - ACTIVE STATIONS
# - NUMBER OF TIMESTAMPS
def add_to_meta_table():
    # open closing table file
    # append daily meta DataFrame
    # save file
    pass

In [394]:
# active_stations = prices_data09.station_uuid.unique()
# active_stations_sample = np.random.choice(active_stations, size=100)
# pds = prices_df.query('station_uuid in @active_stations_sample')
# pds



# create a table that carries all stations for each hour of the day

# group by the hour of the day, take the average price if a station is occuring more than once during that time

# if a station occurs, check the *change columns if its a 2 or a 3, and if yes, check if the price is actually different from the previous hour of if prices have just been re-reported

# if prices changed then make a 1 in the price-changed-dummies

# if there are multiple occurences of the same station within one hour, check which prices changed and make en entry for the respective dummy

# if there are multiple occurences of the same station within one hour, for each of the 3 fuel prices, count how often it changed

# take a batch for each hour of the day

# check 

# df = pd.merge(df, pds, how='left', on=['date', 'station_uuid']).set_index(['date', 'station_uuid'])